In [1]:
# Standard library imports
import logging
import warnings
from pathlib import Path

# Third-party imports
import pandas as pd
import wrds

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

from settings import config
from pull_compustat import *
from pull_crsp import *
from transform_crsp import *
from transform_compustat import *

In [2]:
# Change default pandas display options

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 200
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.expand_frame_repr', False)

# Global variables
RAW_DATA_DIR = Path(config("RAW_DATA_DIR"))
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

In [3]:
crsp_d = pull_CRSP_stock(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    freq='D',
    data_dir=RAW_DATA_DIR,
    file_name='CRSP_stock_d.parquet',
)

Loading cached data from C:\Users\singe\OneDrive - The University of Chicago\Documents\GitHub\FM-ReturnPrediction\_data\raw\CRSP_stock_d.parquet


In [4]:
crsp_m = pull_CRSP_stock(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    freq='M',
    data_dir=RAW_DATA_DIR,
    file_name='CRSP_stock_m.parquet',
)

Loading cached data from C:\Users\singe\OneDrive - The University of Chicago\Documents\GitHub\FM-ReturnPrediction\_data\raw\CRSP_stock_m.parquet


In [5]:
comp = pull_Compustat(
    start_date=START_DATE,
    end_date=END_DATE,
    wrds_username=WRDS_USERNAME,
    data_dir=RAW_DATA_DIR,
    file_name='Compustat_fund.parquet',
)

Loading cached data from C:\Users\singe\OneDrive - The University of Chicago\Documents\GitHub\FM-ReturnPrediction\_data\raw\Compustat_fund.parquet


In [6]:
ccm = pull_CRSP_Comp_link_table(
    wrds_username=WRDS_USERNAME,
    data_dir=RAW_DATA_DIR,
    file_name="CRSP_Comp_Link_Table.parquet"
    )

Loading cached data from C:\Users\singe\OneDrive - The University of Chicago\Documents\GitHub\FM-ReturnPrediction\_data\raw\CRSP_Comp_Link_Table.parquet


In [7]:
crsp_index_d = pull_CRSP_index(
    start_date=START_DATE,
    end_date=END_DATE,
    freq='D',
    wrds_username=WRDS_USERNAME,
    file_name="CRSP_index_d.parquet")

Loading cached data from C:\Users\singe\OneDrive - The University of Chicago\Documents\GitHub\FM-ReturnPrediction\_data\raw\CRSP_index_d.parquet


In [8]:
# 2) Calculate market equity
crsp = calculate_market_equity(crsp_m)

# 2) Add report date and calculate book equity
comp = add_report_date(comp)
comp = calc_book_equity(comp)
comp = expand_compustat_annual_to_monthly(comp)


In [9]:
# 3) Merge comp + crsp_m + ccm => crsp_comp
crsp_comp = merge_CRSP_and_Compustat(crsp, comp, ccm)

In [10]:
from calc_Lewellen_2014 import *

In [11]:
crsp_comp          = calc_log_size(crsp_comp)

In [12]:
crsp_comp            = calc_log_bm(crsp_comp)

In [13]:
crsp_comp       = calc_return_12_2(crsp_comp)

In [14]:
crsp_comp          = calc_accruals(crsp_comp) 

In [15]:
crsp_comp               = calc_roa(crsp_comp)

In [16]:
crsp_comp = calc_log_assets_growth(crsp_comp)

In [17]:
crsp_comp                = calc_dy(crsp_comp)

In [18]:
crsp_comp  = calc_log_return_13_36(crsp_comp)

In [19]:
crsp_comp     = calc_log_issues_12(crsp_comp)

In [20]:
crsp_comp     = calc_log_issues_36(crsp_comp)

In [21]:
crsp_comp        = calc_debt_price(crsp_comp)

In [22]:
crsp_comp       = calc_sales_price(crsp_comp)

In [23]:
crsp_comp = calc_std_12(crsp_d, crsp_comp)


In [24]:
crsp_comp = calculate_rolling_beta(crsp_d, crsp_index_d, crsp_comp)

In [26]:
# Winsorize the variables to remove outliers
variables_dict = {
    "Return (%)":                "retx",                # Assuming you are keeping this column name
    "Log Size (-1)":             "log_size",
    "Log B/M (-1)":              "log_bm",
    "Return (-2, -12)":          "return_12_2",
    "Log Issues (-1,-12)":       "log_issues_12",
    "Accruals (-1)":             "accruals_final",
    "ROA (-1)":                  "roa",
    "Log Assets Growth (-1)":    "log_assets_growth",
    "Dividend Yield (-1,-12)":   "dy",
    "Log Return (-13,-36)":      "log_return_13_36",
    "Log Issues (-1,-36)":       "log_issues_36",
    "Beta (-1,-36)":             "beta",
    "Std Dev (-1,-12)":          "rolling_std_252",
    "Debt/Price (-1)":           "debt_price",
    "Sales/Price (-1)":          "sales_price",
    }
crsp_comp = winsorize(crsp_comp, variables_dict.values())

In [ ]:

def get_subsets(crsp_comp: pd.DataFrame) -> dict:
    """
    Given a monthly CRSP DataFrame with columns at least:
       ['mthcaldt', 'permno', 'me', 'primaryexch'],
    compute the NYSE 20th and 50th percentile of 'me' each month, store them
    in each row as 'me_20' and 'me_50', then build subset DataFrames:

      1) all_stocks          : everyone
      2) all_but_tiny_stocks : rows where me >= me_20
      3) large_stocks        : rows where me >= me_50

    If a particular month has no NYSE stocks (so me_20 or me_50 is NaN),
    then no rows from that month go into the 'all_but_tiny_stocks' or
    'large_stocks' subsets.

    Returns
    -------
    dict
        {
          "all_but_tiny_stocks":  <DataFrame of rows with me >= me_20>,
          "large_stocks":         <DataFrame of rows with me >= me_50>,
          "all_stocks":           crsp_comp   (the entire dataset)
        }
    """
    # 1) Sort for consistent grouping
    crsp_comp = crsp_comp.sort_values(["mthcaldt", "permno"]).copy()

    # 2) Compute month-specific me_20 and me_50 from NYSE
    #    group by mthcaldt, restrict to primaryexch == 'N'
    #    then get quantile(0.2) and quantile(0.5)
    nyse_me_percentiles = (
        crsp_comp
        .loc[crsp_comp["primaryexch"] == "N"]      # keep only NYSE rows
        .groupby("mthcaldt")["me"]
        .quantile([0.2, 0.5])                      # get 20th & 50th
        .unstack(level=1)                          # pivot so columns = [0.2, 0.5]
        .reset_index()
        .rename(columns={0.2: "me_20", 0.5: "me_50"})
    )
    # nyse_stats has columns ['mthcaldt', 'me_20', 'me_50']

    # 3) Merge these percentile columns back to crsp_comp
    crsp_comp = pd.merge(
        crsp_comp,
        nyse_me_percentiles,
        on="mthcaldt",
        how="left"
    )

    # 4) Create boolean columns for "all_but_tiny" and "large"
    #    If me_20 or me_50 is NaN (month has no NYSE?), these will be False
    crsp_comp["is_all_but_tiny"] = crsp_comp["me"] >= crsp_comp["me_20"]
    crsp_comp["is_large"]        = crsp_comp["me"] >= crsp_comp["me_50"]

    # 5) Now build the dictionary of DataFrames
    all_stocks_df = crsp_comp.copy()

    # For "all_but_tiny", we keep only rows with is_all_but_tiny == True
    all_but_tiny_df = crsp_comp.loc[crsp_comp["is_all_but_tiny"] == True].copy()

    # For "large_stocks", keep only rows with is_large == True
    large_stocks_df = crsp_comp.loc[crsp_comp["is_large"] == True].copy()

    subsets_crsp_comp = {
        "all_stocks":          all_stocks_df,
        "all_but_tiny_stocks": all_but_tiny_df,
        "large_stocks":        large_stocks_df,
    }
    return subsets_crsp_comp

In [47]:
subsets_comp_crsp = get_subsets(crsp_comp) 

In [48]:
def build_table_1(subsets_crsp_comp: dict,
                  variables_dict: dict) -> pd.DataFrame:
    """
    For each variable in `variables_dict`, compute monthly cross-sectional
    (mean, std, count) and then time-series average those stats. We'll do it
    for each subset in `subsets_crsp_comp`.

    Parameters
    ----------
    subsets_crsp_comp : dict
        {
          "all_stocks":          <DataFrame>,
          "all_but_tiny_stocks": <DataFrame>,
          "large_stocks":        <DataFrame>
        }
    variables_dict : dict
        Example:
        {
          "Return (%)": "retx",
          "Log Size (-1)": "log_size",
          "Log B/M (-1)":  "log_bm",
           ...
        }

    Returns
    -------
    pd.DataFrame
       One row per variable in `variables_dict`,
       columns = [<subset>_Mean, <subset>_Std, <subset>_N] for each subset.
    """
    results = []

    for var_label, var_col in variables_dict.items():
        row_stats = []

        # For each subset DataFrame in subsets_crsp_comp
        for subset_name, df_subset in subsets_crsp_comp.items():
            # 1) Filter out nulls
            df_var = df_subset.dropna(subset=[var_col])
            if df_var.empty:
                # If none, fill with NaN
                row_stats.extend([np.nan, np.nan, np.nan])
                continue

            # 2) Group by month, compute cross-sectional stats
            monthly_stats = df_var.groupby("mthcaldt")[var_col].agg(["mean", "std", "count"])

            # 3) Time-series average across months
            avg_mean = monthly_stats["mean"].mean()
            avg_std  = monthly_stats["std"].mean()
            avg_n    = monthly_stats["count"].mean()

            row_stats.extend([avg_mean, avg_std, avg_n])

        # We'll store a tuple of: (var_label, subset1 stats, subset2 stats, ...)
        results.append((var_label, *row_stats))

    # Build columns
    columns = ["Variable"]
    for subset_name in subsets_crsp_comp.keys():
        columns += [
            f"{subset_name}_Mean",
            f"{subset_name}_Std",
            f"{subset_name}_N"
        ]

    summary_df = pd.DataFrame(results, columns=columns)
    return summary_df

In [49]:
table_1 = build_table_1(subsets_comp_crsp, variables_dict)

In [50]:
table_1

,Variable,all_stocks_Mean,all_stocks_Std,all_stocks_N,all_but_tiny_stocks_Mean,all_but_tiny_stocks_Std,all_but_tiny_stocks_N,large_stocks_Mean,large_stocks_Std,large_stocks_N
0,Return (%),0.0085,0.1582,4055.1071,0.0210,0.1182,2175.1250,0.0209,0.0996,1178.8929
1,Log Size (-1),13.5281,2.3682,4017.5273,15.2922,1.4156,2156.3091,16.2855,1.1021,1171.2000
2,Log B/M (-1),-7.5670,1.3216,4017.5273,-7.9505,1.1984,2156.3091,-8.2327,1.1468,1171.2000
3,"Return (-2, -12)",0.1131,0.6467,3547.1818,0.2086,0.5470,1939.2045,0.2099,0.5258,1077.9091
4,"Log Issues (-1,-12)",-0.0098,0.3937,3547.9545,0.0253,0.1542,1939.4091,0.0185,0.1326,1077.9773
5,Accruals (-1),-883.9598,3480.2100,3298.9464,-1566.2511,4521.0902,1731.6786,-2459.8581,5557.4966,955.1786
6,ROA (-1),-0.0955,0.3012,4045.4107,0.0072,0.1470,2172.4643,0.0345,0.1165,1178.1964
7,Log Assets Growth (-1),0.0795,0.3126,3547.9545,0.1040,0.2357,1939.4091,0.1068,0.2133,1077.9773
8,"Dividend Yield (-1,-12)",38.0175,143.6514,4003.0727,60.8475,176.1853,2146.9818,83.7763,202.6555,1165.4727
9,"Log Return (-13,-36)",-0.2443,0.8244,1929.8500,0.0073,0.5353,1123.3000,0.0435,0.4891,650.4000
